In [1]:
import json

import pandas as pd
from pandas.io.json import json_normalize

## Test data

In [2]:
d0 = {'int': 42, 'string': 'answer', 'float': 3.14, 'list': [10, 20, 30]}
d0

{'int': 42, 'string': 'answer', 'float': 3.14, 'list': [10, 20, 30]}

In [3]:
d = {
    'dict': {'a': 1, 'b': 2},
    'list_of_dict': [{'a': 1, 'b': 2, 'dict': {'a': 3, 'b': 4}},
                     {'a': 5, 'b': 6, 'dict': {'a': 7, 'b': 8}}],
}
d.update(d0)
d

{'dict': {'a': 1, 'b': 2},
 'list_of_dict': [{'a': 1, 'b': 2, 'dict': {'a': 3, 'b': 4}},
  {'a': 5, 'b': 6, 'dict': {'a': 7, 'b': 8}}],
 'int': 42,
 'string': 'answer',
 'float': 3.14,
 'list': [10, 20, 30]}

In [4]:
json.dumps(d0)

'{"int": 42, "string": "answer", "float": 3.14, "list": [10, 20, 30]}'

In [5]:
json.dumps(d)

'{"dict": {"a": 1, "b": 2}, "list_of_dict": [{"a": 1, "b": 2, "dict": {"a": 3, "b": 4}}, {"a": 5, "b": 6, "dict": {"a": 7, "b": 8}}], "int": 42, "string": "answer", "float": 3.14, "list": [10, 20, 30]}'

## Built-in `read_json()`

Can expand lists into multiple rows, but does not work with more complicated dicts.  
The expected structure is set by orient parameter.

In [6]:
pd.read_json(json.dumps(d0))

,int,string,float,list
0,42,answer,3.14,10
1,42,answer,3.14,20
2,42,answer,3.14,30


In [7]:
pd.read_json(json.dumps(d0), orient='index')

,0
float,3.14
int,42
list,"[10, 20, 30]"
string,answer


In [8]:
orient = ['split', 'records', 'index', 'columns', 'values', 'table']
for o in orient:
    try:
        pd.read_json(json.dumps(d), orient=o)
    except Exception as exc:
        print(exc)

JSON data had unexpected key(s): string, dict, list_of_dict, float, int, list
arrays must all be same length
'list' object has no attribute 'items'
arrays must all be same length
arrays must all be same length
'schema'


### Import list of records

No flattening of dictionaries and no expanding of lists!

In [9]:
pd.read_json(json.dumps([d0]))

,int,string,float,list
0,42,answer,3.14,"[10, 20, 30]"


In [10]:
pd.read_json(json.dumps([d]))

,dict,list_of_dict,int,string,float,list
0,"{'a': 1, 'b': 2}","[{'a': 1, 'b': 2, 'dict': {'a': 3, 'b': 4}}, {...",42,answer,3.14,"[10, 20, 30]"


## Built-in `json_normalize()`

Expands dicts, but not lists.

In [11]:
json_normalize(d0)

,int,string,float,list
0,42,answer,3.14,"[10, 20, 30]"


In [12]:
json_normalize(d)

,list_of_dict,int,string,float,list,dict.a,dict.b
0,"[{'a': 1, 'b': 2, 'dict': {'a': 3, 'b': 4}}, {...",42,answer,3.14,"[10, 20, 30]",1,2


## Manual flattening

In [13]:
df = pd.DataFrame([d] * 3)
df

,dict,list_of_dict,int,string,float,list
0,"{'a': 1, 'b': 2}","[{'a': 1, 'b': 2, 'dict': {'a': 3, 'b': 4}}, {...",42,answer,3.14,"[10, 20, 30]"
1,"{'a': 1, 'b': 2}","[{'a': 1, 'b': 2, 'dict': {'a': 3, 'b': 4}}, {...",42,answer,3.14,"[10, 20, 30]"
2,"{'a': 1, 'b': 2}","[{'a': 1, 'b': 2, 'dict': {'a': 3, 'b': 4}}, {...",42,answer,3.14,"[10, 20, 30]"


### Select columns by type

In [14]:
df.iloc[0].apply(type)

dict                     <class 'dict'>
list_of_dict             <class 'list'>
int               <class 'numpy.int64'>
string                    <class 'str'>
float           <class 'numpy.float64'>
list                     <class 'list'>
Name: 0, dtype: object

In [15]:
df.iloc[0].apply(type) == list

dict            False
list_of_dict     True
int             False
string          False
float           False
list             True
Name: 0, dtype: bool

### Flatten dicts

In [16]:
df['dict']

0    {'a': 1, 'b': 2}
1    {'a': 1, 'b': 2}
2    {'a': 1, 'b': 2}
Name: dict, dtype: object

In [17]:
pd.DataFrame(df['dict'].tolist())

,a,b
0,1,2
1,1,2
2,1,2


### Flatten lists

In [18]:
df['list']

0    [10, 20, 30]
1    [10, 20, 30]
2    [10, 20, 30]
Name: list, dtype: object

In [19]:
pd.DataFrame(df['list'].apply(enumerate).apply(dict).tolist())

,0,1,2
0,10,20,30
1,10,20,30
2,10,20,30


### Final solution

- iteratively flatten all lists and all dicts
- exclude specified columns from default processing and use custom functions istead

In [20]:
def _expand_column(data: pd.DataFrame, name: str, func, custom_columns: dict, sep: str) -> pd.DataFrame:
    """Split a single column into one or more columns by applying func."""
    new_columns = pd.DataFrame(func(data[name]))

    # prefix column names with parent name
    name_map = {current_name: f'{name}{sep}{current_name}' for current_name in new_columns.columns}

    # add new names to custom columns dict so that they will be processed correctly
    for current_name in set(new_columns.columns) & set(custom_columns):
        new_name = name_map[current_name]
        custom_columns[new_name] = custom_columns[current_name]  # map to the same processing function

    new_columns.rename(name_map, axis=1, inplace=True)

    return new_columns


def expand(data: pd.DataFrame, custom_columns: dict = None, sep: str = '_', depth=100) -> pd.DataFrame:
    """Expand list and dict columns in pandas data frame.

    Args:
        data: Input data frame.
        custom_columns: Dictionary of column names which should be excluded from default processing
            and use custom functions instead.
        sep: Separator to use when merging parent and child names.
        depth: Maximum number of hierarchy levels to expand.
    """
    if data.empty:
        return data

    if custom_columns is None:
        custom_columns = {}

    # expand lists and dicts
    for i in range(depth):
        first_row = data.iloc[0]  # creates a series with column names as index
        column_type = first_row.apply(type)

        list_columns = set(first_row.index[column_type == list]) - set(custom_columns)
        dict_columns = set(first_row.index[column_type == dict]) - set(custom_columns)
        if not list_columns and not dict_columns:
            break
        if i + 1 == depth:
            raise RuntimeError((f'Number of hierarchy levels exceeded, '
                                f'increase depth parameter (currently depth={depth})'))

        new_columns = []
        for name in list_columns:
            new_columns.append(
                _expand_column(data, name,
                               lambda column: column.apply(lambda item: dict(enumerate(item))).tolist(),
                               custom_columns, sep)
            )
        for name in dict_columns:
            new_columns.append(
                _expand_column(data, name, lambda column: column.tolist(), custom_columns, sep)
            )

        remaining_columns = data[set(data.columns) - (list_columns | dict_columns)]
        new_columns.append(remaining_columns)
        data = pd.concat(new_columns, axis=1)

    # process custom columns
    for name, func in custom_columns.items():
        if name in data.columns:
            data[name] = data[name].apply(func)

    return data

In [21]:
expand(df)

,list_of_dict_1_dict_a,list_of_dict_1_dict_b,list_of_dict_0_dict_a,list_of_dict_0_dict_b,string,list_0,list_of_dict_1_a,list_1,list_of_dict_1_b,list_2,list_of_dict_0_b,dict_b,list_of_dict_0_a,dict_a,float,int
0,7,8,3,4,answer,10,5,20,6,30,2,2,1,1,3.14,42
1,7,8,3,4,answer,10,5,20,6,30,2,2,1,1,3.14,42
2,7,8,3,4,answer,10,5,20,6,30,2,2,1,1,3.14,42


In [22]:
expand(df, custom_columns={'dict': str})

,list_of_dict_0_a,list_of_dict_0_b,list_of_dict_0_dict,list_of_dict_1_a,list_of_dict_1_b,list_of_dict_1_dict,string,dict,list_0,list_1,list_2,float,int
0,1,2,"{'a': 3, 'b': 4}",5,6,"{'a': 7, 'b': 8}",answer,"{'a': 1, 'b': 2}",10,20,30,3.14,42
1,1,2,"{'a': 3, 'b': 4}",5,6,"{'a': 7, 'b': 8}",answer,"{'a': 1, 'b': 2}",10,20,30,3.14,42
2,1,2,"{'a': 3, 'b': 4}",5,6,"{'a': 7, 'b': 8}",answer,"{'a': 1, 'b': 2}",10,20,30,3.14,42


### Benchmark

In [23]:
%%timeit
expand(pd.DataFrame([d]), custom_columns={'dict': str})

14.5 ms ± 723 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%%timeit
expand(pd.DataFrame([d] * 10000), custom_columns={'dict': str})

99.4 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
